In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
from sklearn.feature_selection import SelectFromModel

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb

In [2]:
train_data = pd.read_csv('clean_train_data.csv')
test_data = pd.read_csv('clean_test_data.csv')
future_data = pd.read_csv('clean_future_test_data.csv')

### Logistic Regression

In [3]:
X_train = train_data.drop(columns=['id', 'home_team_win', 'season'])
y_train = train_data['home_team_win']
X_test = future_data.drop(columns=['id', 'season'])

In [4]:
def lr_evaluate(C, penalty, solver):
    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver=solver,
        max_iter=1000,
        random_state=42,
    )
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    return scores.mean()
pbounds = {
    'C': (0.01, 10),  # 正則化強度，C 越小正則化越強
    'penalty': (0, 1),  # 0 表示 'l1'，1 表示 'l2'
    'solver': (0, 1),  # 0 表示 'liblinear'，1 表示 'saga'
}

penalty_map = {0: 'l1', 1: 'l2'}
solver_map = {0: 'liblinear', 1: 'saga'}

def wrapped_lr_evaluate(C, penalty, solver):
    penalty = penalty_map[int(round(penalty))]
    solver = solver_map[int(round(solver))]
    return lr_evaluate(C, penalty, solver)

optimizer = BayesianOptimization(
    f=wrapped_lr_evaluate,
    pbounds=pbounds,
    random_state=42,
)

optimizer.maximize(init_points=5, n_iter=20)

best_params = optimizer.max['params']
best_penalty = penalty_map[int(round(best_params['penalty']))]
best_solver = solver_map[int(round(best_params['solver']))]

print("Best_param:")
print(f"C: {best_params['C']:.4f}")
print(f"penalty: {best_penalty}")
print(f"solver: {best_solver}")

LR_model = LogisticRegression(
    C=best_params['C'],
    penalty=best_penalty,
    solver=best_solver,
    max_iter=1000,
    random_state=42,
)
LR_model.fit(X_train, y_train)
print("模型訓練完成！")

|   iter    |  target   |     C     |  penalty  |  solver   |
-------------------------------------------------------------


C:\Users\Liao\anaconda3\Lib\site-packages\bayes_opt\target_space.py:344: UserWarning: You're attempting to register an np.ndarray. Currently, the optimizer internally sorts parameters by key and expects any registered array to respect this order. In future versions this behaviour will change and the order as given by the pbounds dictionary will be used. If you wish to retain sorted parameters, please manually sort your pbounds dictionary before constructing the optimizer.
  warn(msg, stacklevel=1)


| 1         | 0.5511    | 3.752     | 0.9507    | 0.732     |
| 2         | 0.5511    | 5.991     | 0.156     | 0.156     |
| 3         | 0.5512    | 0.5903    | 0.8662    | 0.6011    |
| 4         | 0.5509    | 7.084     | 0.02058   | 0.9699    |
| 5         | 0.551     | 8.326     | 0.2123    | 0.1818    |
| 6         | 0.5512    | 0.6466    | 0.846     | 0.6075    |
| 7         | 0.5513    | 0.1722    | 0.01481   | 0.02931   |
| 8         | 0.5566    | 0.04734   | 0.02316   | 0.9865    |
| 9         | 0.5504    | 0.3525    | 0.01743   | 0.9355    |
| 10        | 0.5566    | 0.05671   | 0.08623   | 0.9227    |
| 11        | 0.5597    | 0.03257   | 0.176     | 0.9837    |
| 12        | 0.5595    | 0.03519   | 0.4139    | 0.8849    |
| 13        | 0.5531    | 0.1383    | 0.4238    | 0.9984    |
| 14        | 0.5601    | 0.02488   | 0.3216    | 0.8526    |
| 15        | 0.5596    | 0.01413   | 0.412     | 0.7261    |
| 16        | 0.5514    | 3.09      | 0.8611    | 0.09384   |
| 17    

### Random Forest

In [5]:
def rf_evaluate(n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features):
    model = RandomForestClassifier(
        n_estimators=int(n_estimators),  # 森林中的樹的數量
        max_depth=int(max_depth),  # 樹的最大深度
        min_samples_split=int(min_samples_split),  # 每個節點的最小分割數量
        min_samples_leaf=int(min_samples_leaf),  # 每個葉子節點的最小樣本數量
        max_features=max_features,  # 每次分割考慮的最大特徵數量
        random_state=42,
        n_jobs=-1  # 使用所有 CPU 核心加速計算
    )
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    return scores.mean()

pbounds = {
    'n_estimators': (50, 300),  # 樹的數量範圍
    'max_depth': (5, 20),  # 樹的最大深度範圍
    'min_samples_split': (2, 20),  # 最小分割樣本數量
    'min_samples_leaf': (1, 10),  # 最小葉子樣本數量
    'max_features': (0.1, 1.0),  # 最大特徵數比例
}

optimizer = BayesianOptimization(
    f=rf_evaluate,
    pbounds=pbounds,
    random_state=42,
)

optimizer.maximize(init_points=5, n_iter=10)

best_params = optimizer.max['params']
print("Best_param:")
print(f"n_estimators: {int(best_params['n_estimators'])}")
print(f"max_depth: {int(best_params['max_depth'])}")
print(f"min_samples_split: {int(best_params['min_samples_split'])}")
print(f"min_samples_leaf: {int(best_params['min_samples_leaf'])}")
print(f"max_features: {best_params['max_features']:.4f}")

RF_model = RandomForestClassifier(
    n_estimators=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']),
    min_samples_split=int(best_params['min_samples_split']),
    min_samples_leaf=int(best_params['min_samples_leaf']),
    max_features=best_params['max_features'],
    random_state=42,
    n_jobs=-1,
)
RF_model.fit(X_train, y_train)
print("模型訓練完成！")

|   iter    |  target   | max_depth | max_fe... | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------


C:\Users\Liao\anaconda3\Lib\site-packages\bayes_opt\target_space.py:344: UserWarning: You're attempting to register an np.ndarray. Currently, the optimizer internally sorts parameters by key and expects any registered array to respect this order. In future versions this behaviour will change and the order as given by the pbounds dictionary will be used. If you wish to retain sorted parameters, please manually sort your pbounds dictionary before constructing the optimizer.
  warn(msg, stacklevel=1)


| 1         | 0.5427    | 10.62     | 0.9556    | 7.588     | 12.78     | 89.0      |
| 2         | 0.5567    | 7.34      | 0.1523    | 8.796     | 12.82     | 227.0     |
| 3         | 0.554     | 5.309     | 0.9729    | 8.492     | 5.822     | 95.46     |
| 4         | 0.5534    | 7.751     | 0.3738    | 5.723     | 9.775     | 122.8     |
| 5         | 0.5465    | 14.18     | 0.2255    | 3.629     | 8.595     | 164.0     |
| 6         | 0.5526    | 7.245     | 0.3037    | 5.217     | 9.648     | 122.2     |
| 7         | 0.5534    | 6.409     | 0.6901    | 4.507     | 10.2      | 224.0     |
| 8         | 0.55      | 8.012     | 0.1987    | 7.61      | 12.18     | 233.1     |
| 9         | 0.5561    | 5.365     | 0.4462    | 7.943     | 16.69     | 225.1     |
| 10        | 0.5521    | 12.16     | 0.8177    | 9.797     | 14.55     | 222.8     |
| 11        | 0.5548    | 6.002     | 0.5329    | 9.818     | 11.65     | 222.0     |
| 12        | 0.5412    | 14.81     | 0.4047    | 9.54

### Support Vector Machine

In [12]:
from sklearn.preprocessing import StandardScaler
best_params = {'C': 6296.748648379015, 'kernel': 'rbf', 'gamma': 'scale'}
SVM_model = SVC(**best_params, probability=True)
SVM_model.fit(X_train, y_train)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
SVM_model.fit(X_train_scaled, y_train)
print("模型訓練完成！")

模型訓練完成！


### XGBoost..


In [9]:
def xgb_evaluate(max_depth, learning_rate, n_estimators, min_child_weight, gamma, subsample, colsample_bytree, alpha, reg_lambda):
    params = {
        'objective': 'binary:logistic',  # 二分类问题
        'eval_metric': 'logloss',  # 评估标准为logloss
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators),
        'min_child_weight': min_child_weight,
        'gamma': gamma,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'alpha': alpha, 
        'reg_lambda': reg_lambda
    }
    
    model = xgb.XGBClassifier(**params)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    return scores.mean()

pbounds = {
    'max_depth': (7, 11),  # 设定max_depth范围
    'n_estimators': (50, 150),  # n_estimators范围
    'subsample': (0.7, 1),  # 样本采样比例范围
    'learning_rate': (0.01, 0.2),  # 学习率范围
    'alpha': (0.0001, 0.2),  # L1正则化系数范围
    'reg_lambda': (0.0001, 0.2),  # L2正则化系数范围
    'gamma': (0, 5),  # gamma范围
    'min_child_weight': (1, 10),  # 子节点的最小样本权重
    'colsample_bytree': (0.3, 1)  # 每棵树的特征采样比例范围
}

optimizer = BayesianOptimization(
    f=xgb_evaluate,  # 目标函数为xgb_evaluate
    pbounds=pbounds,  # 超参数范围
    random_state=42,
)

optimizer.maximize(init_points=5, n_iter=20)

# 获取最佳参数
best_params = optimizer.max['params']
print("Best_param:")
print(f"max_depth: {int(best_params['max_depth'])}")
print(f"learning_rate: {best_params['learning_rate']:.4f}")
print(f"n_estimators: {int(best_params['n_estimators'])}")
print(f"min_child_weight: {best_params['min_child_weight']:.4f}")
print(f"gamma: {best_params['gamma']:.4f}")
print(f"subsample: {best_params['subsample']:.4f}")
print(f"colsample_bytree: {best_params['colsample_bytree']:.4f}")
print(f"alpha: {best_params['alpha']:.4f}")
print(f"lambda: {best_params['reg_lambda']:.4f}")

XGB_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    max_depth=int(best_params['max_depth']),
    learning_rate=best_params['learning_rate'],
    n_estimators=int(best_params['n_estimators']),
    min_child_weight=best_params['min_child_weight'],
    gamma=best_params['gamma'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    alpha=best_params['alpha'],
    reg_lambda=best_params['reg_lambda'],
    random_state=42
)

XGB_model.fit(X_train, y_train)

print("XGBoost模型训练完成！")


|   iter    |  target   |   alpha   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------


C:\Users\Liao\anaconda3\Lib\site-packages\bayes_opt\target_space.py:344: UserWarning: You're attempting to register an np.ndarray. Currently, the optimizer internally sorts parameters by key and expects any registered array to respect this order. In future versions this behaviour will change and the order as given by the pbounds dictionary will be used. If you wish to retain sorted parameters, please manually sort your pbounds dictionary before constructing the optimizer.
  warn(msg, stacklevel=1)


| 1         | 0.5417    | 0.07497   | 0.9655    | 3.66      | 0.1237    | 7.624     | 2.404     | 55.81     | 0.1732    | 0.8803    |
| 2         | 0.5384    | 0.1416    | 0.3144    | 4.85      | 0.1682    | 7.849     | 2.636     | 68.34     | 0.06092   | 0.8574    |
| 3         | 0.5435    | 0.08645   | 0.5039    | 3.059     | 0.0365    | 8.169     | 4.297     | 95.61     | 0.1571    | 0.7599    |
| 4         | 0.5363    | 0.1029    | 0.7147    | 0.2323    | 0.1254    | 7.682     | 1.585     | 144.9     | 0.1931    | 0.9425    |
| 5         | 0.5467    | 0.06099   | 0.3684    | 3.421     | 0.09363   | 7.488     | 5.457     | 53.44     | 0.1819    | 0.7776    |
| 6         | 0.5358    | 0.1069    | 0.4825    | 2.315     | 0.1828    | 8.706     | 6.669     | 53.54     | 0.01565   | 0.7244    |
| 7         | 0.5502    | 0.1522    | 0.809     | 3.142     | 0.02433   | 8.131     | 4.851     | 52.34     | 0.1934    | 0.7631    |
| 8         | 0.5324    | 0.1392    | 0.5884    | 2.935     | 

### Blending

In [13]:
X_train = train_data.drop(columns=['id', 'home_team_win', 'season'])
y_train = train_data['home_team_win']
X_test = future_data.drop(columns=['id', 'season'])

xgb_train_pred = XGB_model.predict_proba(X_train)[:, 1]
rf_train_pred = RF_model.predict_proba(X_train)[:, 1]
svm_train_pred = SVM_model.predict_proba(X_train_scaled)[:, 1]
lr_train_pred = LR_model.predict_proba(X_train)[:, 1]

# 堆疊訓練集預測
stacked_train_predictions = np.column_stack((xgb_train_pred, rf_train_pred, svm_train_pred, lr_train_pred))

# 訓練堆疊模型
stacked_lr = LogisticRegression(max_iter=1000)
stacked_lr.fit(stacked_train_predictions, y_train)

# 生成測試集概率預測
xgb_test_pred = XGB_model.predict_proba(X_test)[:, 1]
rf_test_pred = RF_model.predict_proba(X_test)[:, 1]
svm_test_pred = SVM_model.predict_proba(X_test_scaled)[:, 1]
lr_test_pred = LR_model.predict_proba(X_test)[:, 1]

# 堆疊測試集預測
stacked_test_predictions = np.column_stack((xgb_test_pred, rf_test_pred, svm_test_pred, lr_test_pred))

# 最終預測
final_predictions = stacked_lr.predict(stacked_test_predictions)
final_predictions = final_predictions.astype(bool)

# 保存結果
output_df = pd.DataFrame({
    'id': future_data['id'],
    'home_team_win': final_predictions
})

output_df.to_csv('blended_predictions.csv', index=False)

print("模型訓練完成，預測結果示例：")
print(output_df.head())

模型訓練完成，預測結果示例：
   id  home_team_win
0   0          False
1   1           True
2   2           True
3   3          False
4   4           True
